# Test of  spr_duallib V0.3 library.

Changes: 
- Matrix management
- Expand Algorithm


In [1]:
from scipy.special import comb

import spr_duallib as dn
e = dn.e
np = dn.np
spr = dn.spr

In [2]:

    

    
    
def int2baseRepr(integer):
    
    #stringbase = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ~!@#$^&*()-_=+[]{}\|:<>.?%`"
    stringbase="0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ~!@#$^&*()-_=+[]{}/|:<>.?%≤≥≠±–œå≈®∑"
    stringbase+="çÇáéíóúÁÉÍÓÚñÑâêîôû°¿£¢¥"
    string = ""
    base = len(stringbase)
    while integer > 0:
      
        (integer,exp) = (integer//(base),integer%(base))
    
        string += stringbase[exp]  
    
    #end while
    return string[::-1]

def baseRepr2int(string):
    
    stringbase="0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ~!@#$^&*()-_=+[]{}/|:<>.?%≤≥≠±–œå≈®∑"
    stringbase+="çÇáéíóúÁÉÍÓÚñÑâêîôû°¿£¢¥"
    
    base = len(stringbase)
    number = 0
    nels = len(string)
    for i in range(nels):
        mult = stringbase.find(string[nels-i-1] )
        number += mult * base ** i
   
    
    #end for
    return number



def getExpArrayBaseRpr(expind,maxorder):
    
    
    # get the string with the exponents.
    string = np.base_repr(expind, base = (maxorder+1))
    #print(string)
    nels = len(string)
    expArray = [0]*nels

    i=1
    for stri in string:
        expArray[nels-i] = int(stri)
        i+=1
        
        
    return expArray
        
    
    
    
def loadDirExpArrayLookUp(maxorder,ndir = 100):
    f = open('srcdoc/d_n' + str(maxorder) + '_m' + str(ndir) + '.dat', 'r')

    line=f.readline()

    nels = int(line.strip())
    lines = f.readlines()
    f.close()
    
    expArrays = [[]]*(nels)
    dirArrays = [[]]*(nels)
    
    Array = [[]]*(nels)
    print("Loading LookUp direction and Exponent Arrays.")
    for i in range(nels):
        newline = lines[i].strip()
        els = newline.split(';')

        expArray = getExpArrayBaseRpr(int(els[0]),maxorder)
        

        dirEls = els[1].split(",")
        ndirels = len(dirEls)
        dirArray = [0]*ndirels

        for j in range(ndirels):
            #dirArray[j] = int(dirEls[j],36)
            dirArray[j] = baseRepr2int(dirEls[j])


        
        Array[i] =[dirArray,expArray] 
        

    print("Loaded ", nels," direction arrays." )
    return Array
    


def binarySearch(array,element,maxorder):
    
    indx = convertDirExp2BaseIndx(element,maxorder)
    
    min_i=0
    max_i=len(array)
    pos = int(min_i+0.5*(max_i-min_i))
    
    
    while (max_i-min_i)>2:
        
        if indx>convertDirExp2BaseIndx(array[pos],maxorder):
      
            min_i=pos
    
        else:
      
            max_i=pos+1
      
        # end if
    
        pos = int(min_i+0.5*(max_i-min_i))
    
    # end while
    
    return pos
    
    
    
def convertDirExp2BaseIndx(Array,maxorder):
    
    num = 0
    dirArray = Array[0]
    expArray = Array[1]
    for i in range(len(dirArray)):
        
        num += expArray[i]*(maxorder+1)**(dirArray[i]-1)
        
    return num
    

# NEW METHODS WITH NUMPY ARRAYS

In [3]:
def genDirExpArrayLookUp(maxorder,ndir = 100):
    
    
    total_combs = dn.findComb(ndir+1,maxorder)
    
    
    f = open('srcdoc/d_n' + str(maxorder) + '_m' + str(ndir) + '.dat', 'w+')
    f.write(str(total_combs-1)+'\n')
    print("Generating a total of ", total_combs-1," directions.")
    indxold = 0
    for mat_indx in range(1,total_combs):
        indx = dn.matIndxToDualIndx(mat_indx,maxorder,initDualIndx = indxold)
        indxold = indx
        dirarr,exparr = dn.getDirArrayExpArray(indx,maxorder)

        # convert exponent array to integer.

        newint = 0
        
        for i in range(len(exparr)):
            
            newint += exparr[i]*(maxorder+1)**i


        newline = str(newint)
        newline += ';'
        for direction in dirarr:
            #newline += np.base_repr(direction,base = 36)
            newline += int2baseRepr(direction)


            newline += ','
        newline = newline[:-1]
        newline += "\n"
        
        f.write(newline)
        
    f.close()
    print('Finished')




def loadDirExpArrayLookUpNumpy(maxorder,ndir = 100):
    
    
    f = open('srcdoc/d_n' + str(maxorder) + '_m' + str(ndir) + '.dat', 'r')

    line=f.readline()

    nels = int(line.strip())
    lines = f.readlines()
    f.close()
    
    
    dirArrays = np.zeros([nels,maxorder],dtype = np.uint16)
    expArrays = np.zeros([nels,maxorder],dtype = np.uint8)
    
    
    print("Loading LookUp direction and Exponent Arrays.")
    for i in range(nels):
        
        newline = lines[i].strip()
        els = newline.split(';')
        #exps = int(els[0])
        
        #expArray = getExpArrayBaseRpr(exps,maxorder)
        

        dirEls = els[1].split(",")
        ndirels = len(dirEls)
              
        # Organize the Exponent Array.
        
        expArray = getExpArrayBaseRprNumpy(int(els[0]),maxorder)
        
        dirArray = np.zeros(ndirels).astype(np.int16)
        #dirArray[0] = 

        for j in range(ndirels):
            #dirArray[j] = int(dirEls[j],36)
            dirArray[j] = baseRepr2int(dirEls[j])


        
        
        # assignment
        
        expArrays[i,:expArray.size] = expArray
        dirArrays[i,:ndirels] = dirArray
        

    print("Loaded ", nels," direction arrays." )
    return (dirArrays,expArrays)





def convertDirExp2BaseIndxNumpy(dirArray,expArray,maxorder):
    
    num = 0
    expArrayObj = np.array(expArray,dtype = object)
    dirArrayObj = np.array(dirArray,dtype = object)

    
    for i in range(len(dirArray)):
        
        if dirArray[i]==0:
        
            break
            
        # end if
        
        num += expArrayObj[i]*(maxorder+1)**(dirArrayObj[i]-1)
        
    # end for
    return num







def getExpArrayBaseRprNumpy(expind,maxorder):
    
    expArray = np.zeros((maxorder,),dtype=np.int8)
    i=0
    while expind !=0  :
        
        exp     = expind%( maxorder+1)
        expind = expind//( maxorder+1)
        
        expArray[i] = exp
        i+=1
        
    return expArray[:i]









def genIndxArray(dirArray,expArray,maxorder):
    
    indxArray = np.zeros((dirArray.shape[0],),dtype = object)
    
    nels = indxArray.size
    
    for i in range(nels):
        
        indxArray[i] = convertDirExp2BaseIndxNumpy(dirArray[i,:],expArray[i,:],maxorder)
        
        
    #end for
    
    return indxArray











def binarySearchNumpy(indxArray,dirArray,expArray,maxorder):
    
    indx = convertDirExp2BaseIndxNumpy(dirArray,expArray,maxorder)
    
    min_i = 0
    max_i = indxArray.size
    pos = int(min_i+0.5*(max_i-min_i))
    
    if indx == 1:
        return 0
    
    while (max_i-min_i)>2:
        
        if indx>indxArray[pos]:
      
            min_i=pos
    
        else:
      
            max_i=pos+1
      
        # end if
    
        pos = int(min_i+0.5*(max_i-min_i))
    
    # end while
    
    return pos

In [4]:
maxorder = 2
ndir = 1000
genDirExpArrayLookUp(maxorder,ndir = ndir)

In [156]:
b = [1,2,3,0,0,0]

a = np.array(b,dtype = np.uint32)

a[np.nonzero(a)]


array([1, 2, 3], dtype=uint32)

In [5]:
maxorder = 2
ndir = 10000
dirArray,expArray = loadDirExpArrayLookUpNumpy(maxorder,ndir=ndir)

Loading LookUp direction and Exponent Arrays.


KeyboardInterrupt: 

In [147]:
print(expArray[:10,:])

[[ 1  0  0  0  0  0  0  0  0  0]
 [ 2  0  0  0  0  0  0  0  0  0]
 [ 3  0  0  0  0  0  0  0  0  0]
 [ 4  0  0  0  0  0  0  0  0  0]
 [ 5  0  0  0  0  0  0  0  0  0]
 [ 6  0  0  0  0  0  0  0  0  0]
 [ 7  0  0  0  0  0  0  0  0  0]
 [ 8  0  0  0  0  0  0  0  0  0]
 [ 9  0  0  0  0  0  0  0  0  0]
 [10  0  0  0  0  0  0  0  0  0]]


In [144]:
convertDirExp2BaseIndxNumpy(dirArray[-1,:],expArray[-1,:],maxorder)

23579476910

In [145]:
indxArray = genIndxArray(dirArray,expArray,maxorder)
indxArray.nbytes

1478040

In [146]:
np.save('dir_n'+str(maxorder)+'_m'+str(ndir),dirArray)
np.save('indx_n'+str(maxorder)+'_m'+str(ndir),indxArray)
np.save('exp_n'+str(maxorder)+'_m'+str(ndir),expArray)

In [77]:
# Efficient As Fuck!

c = np.load('dir_n1_m10000.npy')
print(repr(c[-1,:]))
print(repr(dirArray[-1,:]))

array([34464], dtype=uint16)
array([34464], dtype=uint16)


In [86]:
d = np.load('indx_n1_m100000.npy')

print(d[-1] == indxArray[-1])

True


In [191]:
indxArray.nbytes
for i in range(10):
    print(np.base_repr(indxArray[i],maxorder+1))


1
2
10
11
20
100
101
110
200
1000


In [231]:
# check if all the numbers are correct

for i in range(indxArray.size):
    element = ArrayNumPy[i,:]
    
    calcIndx = binarySearchNumpy(indxArray,element,maxorder)
    
    if calcIndx != i:

        print("\n\nSolution is not good for index i = ",i)
        print("it results in ",calcIndx)


In [232]:
indxArray[0]

1

In [246]:
%timeit binarySearchNumpy(indxArray,ArrayNumPy[250000,:],maxorder)

10000 loops, best of 3: 32.5 µs per loop


In [291]:
b = dn.matIndxToDualIndx(50014998,2)

In [293]:
%timeit dn.matIndxToDualIndx(50014999,2,initDualIndx = 0)


10 loops, best of 3: 28.4 ms per loop


In [247]:
maxorder = 2
Array = loadDirExpArrayLookUp(maxorder,ndir = 1000)

Loading LookUp direction and Exponent Arrays.
Loaded  501500  direction arrays.


In [140]:



maxorder = 4
array = np.array([16,20,1000,0,0]).astype(np.int16)
%timeit convertDirExp2BaseIndxNumpy(array,maxorder)

%timeit convertDirExp2BaseIndx([[20,1000],[1,3]],maxorder)

array.size



10000 loops, best of 3: 21.9 µs per loop
100000 loops, best of 3: 5.23 µs per loop


5

In [149]:
ArrayNumPy.nbytes

3009000

In [133]:





%timeit getExpArrayBaseRprNumpy(82906087075,35)
%timeit getExpArrayBaseRpr(82906087075,35)




The slowest run took 4.51 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 4.37 µs per loop
100000 loops, best of 3: 6.95 µs per loop
The slowest run took 4.10 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 7.32 µs per loop


In [248]:
maxorder = 4

%timeit convertDirExp2BaseIndx([[20,1000],[1,3]],maxorder)

100000 loops, best of 3: 5.04 µs per loop


In [343]:
binarySearch(Array,element,maxorder)

4587924

In [344]:
binarySearch(Array,element,maxorder) == Array.index(element)

True

In [345]:
convertDirExp2BaseIndx(element,maxorder)

1893266172530428332988148556678686951215695444261655211448669433593750

In [274]:
int('1'*6,6)

137257

In [161]:
ab = np.array([])
ab

0